In [1]:
from pyspark.sql import DataFrame,Row
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType
from pyspark.sql.functions import from_json, col

In [2]:
spark = SparkSession.builder.appName('my_first_app_name_1').getOrCreate()

In [3]:
jstr1 = u'{"header":{"id":12345,"foo.cn":"bar"}}'
jstr2 = u'{"header"{"id":12345,"foo.cn":"bar"}}'
jstr3 = u'{"header":{"id":12345,"foo.cn":"bar"}}'
df = spark.createDataFrame([Row(json=jstr1),Row(json=jstr2),Row(json=jstr3)])

In [4]:
new_df = spark.read.json(df.rdd.map(lambda r: r.json))

In [ ]:
new_df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- header: struct (nullable = true)
 |    |-- foo.cn: string (nullable = true)
 |    |-- id: long (nullable = true)



In [160]:
json_schema = new_df.schema

In [163]:
df = df.withColumn('name', from_json(col("json"), json_schema))

In [164]:
df.show()

+--------------------+----------------+
|                json|            name|
+--------------------+----------------+
|{"header":{"id":1...|[, [bar, 12345]]|
|{"header"{"id":12...|            null|
|{"header":{"id":1...|[, [bar, 12345]]|
+--------------------+----------------+



In [188]:
new_select_df = df.select(df['name.header'].alias('header'),df['json']).select('header.*','json')

In [189]:
new_select_df.show()

+------+-----+--------------------+
|foo.cn|   id|                json|
+------+-----+--------------------+
|   bar|12345|{"header":{"id":1...|
|  null| null|{"header"{"id":12...|
|   bar|12345|{"header":{"id":1...|
+------+-----+--------------------+



In [191]:
new_select_df = new_select_df.select(new_select_df['`foo.cn`'].alias('foo_cn'))
new_select_df.show()

+------+
|foo_cn|
+------+
|   bar|
|  null|
|   bar|
+------+



In [ ]:
########################### 如果想要把new_select_df 的列附加到df 上是不可行的，不同的dataframe 用withColumn 会报异常################################

AnalysisException: "cannot resolve '`data`' given input columns: [json, name];;\n'Project [json#536, name#545, jsontostructs(StructField(start.date,StringType,true), StructField(end.date,StringType,true), 'data, Some(Asia/Shanghai)) AS data#584]\n+- Project [json#536, jsontostructs(StructField(header,StructType(StructField(foo.cn,StringType,true), StructField(id,LongType,true)),true), json#536, Some(Asia/Shanghai)) AS name#545]\n   +- LogicalRDD [json#536], false\n"